# with index, and link graph

## rank graph  
{'url':[pages links]}

In [2]:
def crawl_web(seed, max_depth):
    tocrawl = [seed]
    crawled = []
    next_depth = []
    depth = 0
    index = {}
    graph = {}
    while tocrawl and depth <= max_depth:
        page = tocrawl.pop()
        if page not in crawled:
            content = get_page(page)
            add_page_to_index(index, page, content)
            outlinks = get_all_links()
            graph[page] = outlinks
            union(next_depth, outlinks)
            crawled.append(page)
        if not tocrawl:
            tocrawl, next_depth = next_depth, []
            depth = depth + 1
    return crawled 

## popularity

``
popularity(p) = \# of people who are friends with p
``

$$
popularity(p) = \sum{popularity(f)}
$$

```
def popularity(p):
    score = 0
    for f in friends(p):
        score = score + popularity(f)
    return score
```

### circular defination!
### 循环了，如何退出
定义base case, 
下面是否正确？
```
def popularity(p):
    if p == 'Alice': 
        return 1
    score = 0
    for f in friends(p):
        score = score + popularity(f)
    return score
```

## Relaxation Algorithm
``` 
 start with a guss
 while not done:
    make the guess better
```    
 popularity(0,p) -> 1 <br/>
 popularity(t,p) = $\displaystyle\sum_{f\in{friends(p)}}{ popularity(t-1,f) }$
```
def popularity(t, p):
    if t == 0:
        return 1
    else:
        score = 0
        for f in friends(p):
            score = score + popularity(t-1, f)
        return score    
```

URank<br/>
$rank(0,url) -> 1$ <br/>
$rank(t, url) -> \displaystyle\sum_{p\in{inlinks[url]}}rank(t-1,p)$

if too much outlinks: <br/>
$rank(t, url) -> \frac{\displaystyle\sum_{p\in{inlinks[url]}}rank(t-1,p)}{outlinks[p]}$


<p>我们随机浏览网页时候，也会决定再随机看一个新网页，而不是一直再一个网站中浏览下去，d (damping value)就是代码我们认为用户 
选择就像顺着link路径浏览下去 vs 选择一个新网站</p>
<p>理解阻尼系数，PR值衡量的其实是某个页面被访问的可能性的大小，链接到它的网页越多，通过这些链接点开它的可能性当然就越大，但在没有任何外部链接的情况下，一个网页是不是就不可能被访问呢？当然不是，因为还可以从地址框直接键入url访问，而且也会有人希望通过搜索引擎找到这个页面，所以我们在公式里加了阻尼系数，当然这里给1-d取一个比较小的数才可以。</p>

d = 0.8 <br/> 
N = number of pages<br/>

$rank(0,url) -> 1 / N $ <br/>

$rank(t, url) -> d \dot \displaystyle\sum_{p\in{inlinks[url]}}rank(t-1,p) / outlinks[p]) + \frac{1-d}{N}$

In [3]:
ex_graph={
       'a':['b''c','d'],
       'b':[],
       'c':['a'],
       'd':[]}

In [6]:
def compute_ranks(graph):
    d = 0.8
    numloops = 10
    
    ranks = {}
    npages = len(graph)
    for page in graph:
        ranks[page] = 1.0 / npages
        
    for i in range(0, numloops):    
        newranks = {}
        for page in graph:
            newrank = (1-d)/npages
            # update by summing in the inlink ranks
            for node in graph:
                if page in graph[node]:
                    newrank = newrank + d * (ranks[node]/len(graph[node]))
            newranks[page] = newrank
        ranks = newranks
    return ranks

        

In [7]:
result = compute_ranks(ex_graph)

In [8]:
result

{'a': 0.08999999999999998,
 'b': 0.04999999999999999,
 'c': 0.04999999999999999,
 'd': 0.086}